# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [ ]:
task = '''
       Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: Unveiling the Power of DeepLearning.AI

Delve into the world of artificial intelligence with DeepLearning.AI, a leading platform revolutionizing the field. From introductory courses to specialized programs, DeepLearning.AI offers a wide range of resources to sharpen your skills and propel your career forward. With expert-led instruction and hands-on projects, you'll gain valuable insights into deep learning, neural networks, and much more. Whether you're a novice or an experienced professional, there's something for everyone to learn and explore. Join the DeepLearning.AI community today and unlock the doors to endless possibilities in the realm of AI.


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of DeepLearning.AI

Delve into the world of artificial intelligence with DeepLearning.AI, a leading platform revolutionizing the field. From introductory courses to specialized programs, DeepLearning.AI offers a wide range of resources to sharpen your skills and propel your career forward. With expert-led instruction and hands-on projects, you'll gain valuable insights into deep learning, neural networks, and much more. Whether you're a novice or an experienced professional, there's something for everyone to learn and explore. Join the DeepLearning.AI community today and unlock the doors to endless possibilities in the realm of AI.

------------------------------------------------------------------------

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of DeepLearning.AI

Delve into the world of artificial intelligence with DeepLearning.AI, a leading platform revolutionizing the field. From introductory courses to specialized programs, DeepLearning.AI offers a wide range of resources to sharpen your skills and propel your career forward. With expert-led instruction and hands-on projects, you'll gain valuable insights into deep learning, neural networks, and much more. Whether you're a novice or an experienced professional, there's something for everyone to learn and explore. Join the DeepLearning.AI community today and unlock the doors to endless possibilities in the realm of AI.

------------------------------------------------------------------------


--------------------------------------------------------------------------------
Meta Reviewer (to Critic):

After aggregating the feedback from all reviewers, it is clear that the main focus is on optimizing the content for search engine visibility and user engagement. The consistent suggestions include optimizing the title with relevant keywords related to DeepLearning.AI, incorporating related keywords throughout the content, and having a clear call-to-action to encourage users to join the DeepLearning.AI community.

Overall, the reviewers are emphasizing the importance of search engine optimization (SEO) techniques to increase the visibility of the content and drive user engagement. It seems like the main objective is to attract users interested in artificial intelligence courses, deep learning programs, and neural networks to the DeepLearning.AI platform.

Final Suggestion:
1. **Optimize Title**: Ensure that the title includes a keyword or key phrase related to DeepLearning.AI fo

## Get the summary

In [15]:
print(res.summary)

Title: Mastering AI: Your Gateway to DeepLearning.AI

Embark on a transformative journey in artificial intelligence with DeepLearning.AI, a cutting-edge platform reshaping the industry. Curated for all skill levels, from beginners to experts, DeepLearning.AI offers comprehensive courses empowering you with in-depth knowledge of neural networks, deep learning, and more. Elevate your career and skills with expert guidance and hands-on experience. Join the thriving DeepLearning.AI community today and unlock endless opportunities in the AI landscape.

Don't miss out – dive into the world of AI with DeepLearning.AI now!

Incorporate the feedback received from reviewers and deliver a refined version of the blogpost on DeepLearning.AI.
